In [1]:
# External dependencies
import nevergrad as ng
# torch imports
import torch
import numpy as np
import torch_geometric.transforms as T
import torch_geometric.data as Data
from model import Training
from concurrent import futures
# Local dependencies
from data import WikiSigned, Tribes, Chess, BitcoinA

/home/ar/miniconda3/envs/gnn/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parameters

In [6]:
# SpringE parameters
embedding_dim  = 64
time_step = 0.005
iterations = 500
damping = 0.02

# dataset
id = 'bitcoin'
root = 'src/data'
test_size = 0.2

# Setup

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define the transforms
pre_transforms = []
pre_transforms.append(T.ToUndirected())
#transform.append(T.ToSparseTensor())
pre_transforms = T.Compose(pre_transforms)

# Dataset

In [8]:

if id == "wiki":

    dataset = WikiSigned(
        root=root,
        pre_transform=pre_transforms,
        one_hot_signs=False)

elif id == "tribes":
    dataset = Tribes(
        root=root,
        pre_transform=pre_transforms,
        one_hot_signs=False)

elif id == "chess":
    dataset = Chess(
        root=root,
        pre_transform=pre_transforms,
        one_hot_signs=False)

elif id == "bitcoin":
    dataset = BitcoinA(
        root=root,
        pre_transform=pre_transforms)

data = dataset[0]
n_edges = data.edge_index.shape[1]

print(f"Number of edges: {n_edges}")
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of positive edges: {data.edge_attr.sum()}")
print(f"Number of negative edges: {n_edges - data.edge_attr.sum()}")
print(f"Ratio of positive edges: {data.edge_attr.sum() / n_edges}")

# Create train and test datasets
train_data = dataset[0].clone()
test_data = dataset[0].clone()
test_mask =\
    np.random.choice([1, 0], size=n_edges, p=[test_size, 1-test_size])
test_mask = torch.tensor(test_mask)

train_data.edge_attr = torch.where(test_mask == 1, 0, dataset[0].edge_attr)
test_data.edge_attr = torch.where(test_mask == 0, 0, dataset[0].edge_attr)



Number of edges: 28246
Number of nodes: 7605
Number of positive edges: 42226
Number of negative edges: -13980
Ratio of positive edges: 1.4949373006820679


/home/ar/miniconda3/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/storage.py:283: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_attr', 'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


# SpringE Embedding

In [9]:

training = Training(
    device=device,
    train_data=train_data,
    test_data=test_data,
    test_mask=test_mask,
    embedding_dim=embedding_dim,
    time_step=time_step,
    iterations=iterations,
    damping=damping)

parametrization = ng.p.Instrumentation(
    friend_distance=ng.p.Scalar(lower=0.1, upper=15.0),
    friend_stiffness=ng.p.Scalar(lower=0.1, upper=8.0),
    neutral_distance=ng.p.Scalar(lower=0.1, upper=15.0),
    neutral_stiffness=ng.p.Scalar(lower=0.1, upper=8.0),
    enemy_distance=ng.p.Scalar(lower=0.1, upper=15.0),
    enemy_stiffness=ng.p.Scalar(lower=0.1, upper=8.0),
)

optimizer = ng.optimizers.NGOpt(parametrization=parametrization, budget=100, num_workers=1)

recommendation = optimizer.minimize(training)

print(recommendation.value)